In [10]:
%matplotlib widget
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
from signal_alignment import phase_align, chisqr_align
from dateutil.parser import *

In [11]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 10000)

In [12]:
referenceFile = pd.read_csv('./UVVis/BareSolarLamp2.csv')
referenceFile['intensity'] = referenceFile['intensity'].rolling(50, win_type='nuttall').mean().shift(-25).fillna(1)

spectrumBaseline = referenceFile[referenceFile.wavelength > 200]
spectrumBaseline = spectrumBaseline[referenceFile.wavelength < 238]
spectrumBaseline = spectrumBaseline['intensity'].mean()
referenceFile['intensity'] = referenceFile['intensity'] - spectrumBaseline


referenceFileMaxValue = referenceFile[referenceFile.wavelength > 823.15]
referenceFileMaxValue = referenceFileMaxValue[referenceFile.wavelength < 823.359]
referenceFileMaxValue = referenceFileMaxValue['intensity'].mean()

referenceFile['intensityNorm'] = referenceFile['intensity'] / referenceFileMaxValue

referenceFile = referenceFile.set_index('wavelength')

In [13]:
files = [
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure2.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure3.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure4.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure5.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure6.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure7.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure8.csv'
]

dataframes = []

for fileName in files:
    file = pd.read_csv(fileName)
    file['time'] = file['time'].round(0)
    file['time'] = pd.to_datetime(file['time'], unit='s')
    file = file.set_index([pd.DatetimeIndex(file['time'])], drop=True)
    file = file.drop(columns='time')
    dataframes.append(file)
    

file = pd.concat(dataframes)

file = file.drop(columns=['Unnamed: 0', 'n'])
file

,wavelength,intensity,acqTime
time,,,
2021-01-18 13:28:50,178.997010,2754.876723,5699
2021-01-18 13:28:50,179.215281,2754.876723,5699
2021-01-18 13:28:50,179.433542,2754.876723,5699
2021-01-18 13:28:50,179.651791,2687.031412,5699
2021-01-18 13:28:50,179.870030,2738.808097,5699
...,...,...,...
2021-01-18 19:01:29,884.693799,5463.332970,23400
2021-01-18 19:01:29,884.858069,5363.350406,23400
2021-01-18 19:01:29,885.022319,5234.801395,23400


In [14]:
potentiostatData = pd.read_csv('./UVVis/UVVisCell_Sample04_50nmSi_SolarGen/Potentiostate/MergedFile.csv')
# potentiostatData['actualTime'] = potentiostatData['actualTime'].map(lambda x: parse(x).timestamp())
potentiostatData['time'] = pd.to_datetime(potentiostatData['actualTime'])
potentiostatData = potentiostatData.set_index('time')
potentiostatData['elapsedTime'] = potentiostatData.index - potentiostatData.index.values[0]
potentiostatData = potentiostatData.drop(columns=['Unnamed: 0', 'actualTime', 'T'])
potentiostatData

,Vf,Im,elapsedTime
time,,,
2021-01-18 14:29:24.000,3.212300,3.855850e-07,0 days 00:00:00
2021-01-18 14:29:25.000,3.213140,3.139240e-08,0 days 00:00:01
2021-01-18 14:29:26.000,3.212400,3.642480e-09,0 days 00:00:02
2021-01-18 14:29:27.000,3.211840,3.727420e-10,0 days 00:00:03
2021-01-18 14:29:28.000,3.211320,-1.591780e-09,0 days 00:00:04
...,...,...,...
2021-01-19 08:35:29.000,0.009714,-3.758170e-06,0 days 18:06:05
2021-01-19 08:35:29.500,0.009714,-3.756970e-06,0 days 18:06:05.500000
2021-01-19 08:35:30.000,0.009715,-3.756250e-06,0 days 18:06:06


In [15]:
mergedDataframe = pd.merge_asof(file, potentiostatData, left_index = True, right_index = True, tolerance=pd.Timedelta("1 minutes"))
mergedDataframe = mergedDataframe.dropna(how='any')
mergedDataframe

,wavelength,intensity,acqTime,Vf,Im,elapsedTime
time,,,,,,
2021-01-18 14:30:00,178.997010,2826.292840,5581,3.195240,-4.228770e-08,0 days 00:00:36
2021-01-18 14:30:00,179.215281,2826.292840,5581,3.195240,-4.228770e-08,0 days 00:00:36
2021-01-18 14:30:00,179.433542,2826.292840,5581,3.195240,-4.228770e-08,0 days 00:00:36
2021-01-18 14:30:00,179.651791,2874.498720,5581,3.195240,-4.228770e-08,0 days 00:00:36
2021-01-18 14:30:00,179.870030,2854.859287,5581,3.195240,-4.228770e-08,0 days 00:00:36
...,...,...,...,...,...,...
2021-01-18 19:01:29,884.693799,5463.332970,23400,0.356823,-2.735890e-06,0 days 04:32:05
2021-01-18 19:01:29,884.858069,5363.350406,23400,0.356823,-2.735890e-06,0 days 04:32:05
2021-01-18 19:01:29,885.022319,5234.801395,23400,0.356823,-2.735890e-06,0 days 04:32:05


In [21]:
figure2 = plt.figure()

figure2ax1 = figure2.add_subplot(1,2,1)
# figure2ax1.set_ylim(0, 1.5)
figure2ax2 = figure2.add_subplot(1,2,2)
# figure2ax2.set_ylim(-0.5, 6)



X = mergedDataframe.xs(str(mergedDataframe.index.values[0]))['wavelength'].values
Y = [date for date in mergedDataframe.groupby('time').groups]

XX, YY = np.meshgrid(X, Y)

Z_absSmoothed = []
Z_intensityNorm = []
Z_Im = []
Z_V = []

for yGrid in YY:
    try:
        spectrum = mergedDataframe.xs(str(yGrid[0]))
        
        spectrum = spectrum.set_index('wavelength')
        
        spectrum['intensity'] = spectrum['intensity'].rolling(50).mean()

        spectrumBaseline = spectrum[(spectrum.index > 200) & (spectrum.index < 300)]
        spectrumBaseline = spectrumBaseline['intensity'].mean()
        spectrum['intensity'] = spectrum['intensity'] - spectrumBaseline


        spectrumMaxValue = spectrum[(spectrum.index > 823.15) & (spectrum.index < 823.359)]
        spectrumMaxValue = spectrumMaxValue['intensity'].mean()
        spectrum['intensityNorm'] = spectrum['intensity'] / spectrumMaxValue


        spectrum['abs'] = -np.log(spectrum['intensity'] / referenceFile['intensityNorm'])
        spectrum['absSmoothed'] = spectrum['abs'].rolling(50).mean()

        figure2ax1.plot(spectrum.index, spectrum['intensityNorm'])
        figure2ax2.plot(spectrum.index, spectrum['absSmoothed'])

        Z_absSmoothed.append(spectrum['absSmoothed'].values)
        Z_intensityNorm.append(spectrum['intensityNorm'].values)
        Z_Im.append(spectrum['Im'].values)
        Z_V.append(spectrum['Vf'].values)
        
    
    except KeyError as err:
        print(err)
        continue



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
figure3 = plt.figure()

figure3ax1 = figure3.add_subplot(1,4,1)
# ax5.set_xlim(265, 807)

figure3ax2 = figure3.add_subplot(1,4,2)
# ax6.set_xlim(265, 807)

figure3ax3 = figure3.add_subplot(1,4,3)
# ax6.set_xlim(265, 807)

figure3ax4 = figure3.add_subplot(1,4,4)
# ax6.set_xlim(265, 807)

cont = figure3ax1.pcolormesh(XX, YY, Z_absSmoothed, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto', vmin=-12, vmax=0)

cont = figure3ax2.pcolormesh(XX, YY, Z_intensityNorm, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto', vmin=0, vmax=2)

cont = figure3ax3.pcolormesh(XX, YY, Z_V, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto')

cont = figure3ax4.pcolormesh(XX, YY, Z_Im, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …